# Import Liberaries

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import netCDF4 as nc
import xarray as xr

sys.path.append('/home/shirazipooya/w/WaterBalance/')

# Varables

In [ ]:
PATH_DATA = "/mnt/c/Users/shirazipooya/Desktop/DataQDWB/"

# Jovein
LAT_MIN = 36.2
LAT_MAX = 37.0
LON_MIN = 56.2
LON_MAX = 58.6



# Functions

In [ ]:
def mask_nc_file(
    nc,
    variable,
    lat_min,
    lat_max,
    lon_min,
    lon_max
):
    with xr.open_dataset(nc) as xr_nc:
        result = xr_nc.sel(
            lat = slice(lat_max, lat_min),
            lon = slice(lon_min, lon_max),
        )
    return result

# Load Data

In [ ]:
tmax = mask_nc_file(
    nc = PATH_DATA + "Tmax_2022031.nc",
    variable = "air_temperature",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)

tmin = mask_nc_file(
    nc = PATH_DATA + "Tmin_2022031.nc",
    variable = "air_temperature",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)


tmean = mask_nc_file(
    nc = PATH_DATA + "Temp_2022031.nc",
    variable = "air_temperature",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)


### Estimate Reference Crop Evapotranspiration (ETo) Using the Hargreaves and Samani Method.

In [ ]:
from qdwb.evapotranspiration.et import *
from qdwb.evapotranspiration.asset import *
from qdwb.evapotranspiration.convert import *

ra = extraterrestrial_radiation(
    inverse_relative_distance_earth_sun=inverse_relative_distance_earth_sun(
        julian_date=31
    ),
    sunset_hour_angle=sunset_hour_angle(
        latitude=convert_degrees2radians(tmin.lat),
        solar_declination=solar_declination(
            julian_date=31
        )
    ),
    latitude=convert_degrees2radians(
        degrees=tmin.lat
    ),
    solar_declination=solar_declination(
        julian_date=31
    ),
)


ET = ReferenceEvapotranspiration.hargreaves_samani(
    tmin=tmin,
    tmax=tmax,
    tmean=tmean,
    ra=ra
)

ra

In [ ]:
a = tmin > tmax

a.air_temperature.all().values